# Projeto de Séries Temporais com Regressão Linear

## 1. Contexto


---

**O presente** dataset faz parte de uma competição do [Kaggle](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview) do Coursera, [Como Vencer uma Competição de Ciência de Dados](https://www.coursera.org/learn/competitive-data-science/home/welcome). A temática da competição é trabalhar em um projeto de Séries Temporais (TS) com frequência diária de vendas, disponibilizada por uma empresa russa chamada [1C Company](https://1c.ru/eng/title.htm).


**O objetivo** é prever o total de vendas de todos os produtos e história para o próximo mês, levando em consideração o tópico de TS, então temos que fazer essa previsão para 30 períodos a frente. Com isso, temos os seguintes banco de dados e as suas descrições:


a) **sales_train.csv:** Dados históricos de vendas, de Janeiro de 2013 a Outubro 2015

b) **test.csv:** Dados de teste, Novembro de 2015 (os 30 períodos a frente para prever)

c) **items.csv:** Informações adicionais de itens/produtos

d) **item_categories.csv:** Informações adicionais sobre as categorias de item

e) **shops:** Informações adicionais sobre as lojas


**Com** essas informações, criamos um notebook auxiliar para não deixar este relativamente poluído, criando funções, tratando dados, etc. E para isso, utilizamos o comando mágico do jupyter `%run`, para importar os objetos (bancos de dados, funções, etc).

In [22]:
%run function_ts_regressao.ipynb

## 2. Descrição dos dados

---

**Como** mencionado anteriormente, possuem dados em diferentes bancos de dados, podemos ver uma descrição um pouco mais aprofundada de cada um destes. Não iremos utilizar todos os bancos de dados, somente os dados de **sales_train, items, item_categories e shops**.

### 2.1. Sales Train 

i) **date**: data no formato dd/mm/yyyy;

ii) **date_block_num**: Mês consecutivo desde o início até o fim, isto é, 0 corresponde à Janeiro de 2013 e 33 a Outubro de 2015;

iii) **shop_id**: Identificador do shop;

iv) **item_id**: Identificador do item;

v) **item_price**: Preço atual do item;

vi) **item_cnt_day**: Número de produtos vendidos (**target**)

In [23]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [24]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


### 2.2. Items

In [25]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


### 2.3. Item categories

In [26]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


### 2.4. Shops



In [27]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


### 2.5. Conclusões

**Observando** nossos dados, é possível perceber que temos que fazer algumas modificações, como transformação nos tipos, criação de features, etc. Também é perceptível que a maioria dos bancos auxiliares possuem IDs de outras informações para nosso banco de dados principal.

## 3. Definição da métrica

---

**Em regressão**, assim como nos casos de séries temporais, temos métricas bastante consolidadas e utilizáveis na grande maioria dos casos. Em nosso estudo utilizaremos a Raiz Quadrada do Erro Quadrático Médio - Root Mean Squared Error - **RMSE**, que é definida por meio da seguinte fórmula.

$$ \text{RMSE} = \sqrt{\frac{1}{n}\sum_{i}(y_i - \hat{y}_i)^2} $$

Seguindo os passos da equação temos:

1. Subtrair o que o nosso modelo preveu para uma observação *i* pelo valor real desta observação;
2. Para suprir a necessidade de que possa aparecer um valor negativo (dado que seu modelo pode estimar um valor maior que o verdadeiro), elevar ao quadrado o resultado;
3. Levando em consideração que estamos fazendo o processo **1** e **2** para diversas observações, somamos os resultados
4. Após isso, calculamos a média
5. Por fim, utilizamos a raiz quadrada para termos uma métrica interpretável (devido a utilização do quadrado no passo **2**


**Algumas observações sobre o RMSE:**

- Quanto menor, melhor;
- Predições/previsões perfeitas equivalem ao RMSE = 0;
- Penaliza previsões do seu modelo que fogem muito do valor real

Informações retiradas de: [Como avaliar seu modelo de regressão](https://medium.com/turing-talks/como-avaliar-seu-modelo-de-regress%C3%A3o-c2c8d73dab96)

## 4. Modelo de chute

---

A criação de um modelo de chute, ou dummy, serve justamente para termos algo com o que comparar o nosso modelo oficial de séries temporais com regressão linear. Fora que se o nosso modelo de chute for melhor que o modelo oficial, não teria lógica utilizar um modelo mais complexo para prever (princípio da parcimônia). 

Temos uma biblioteca chamada `sktime` com a função `NaiveForecaster`, nela temos alguns modelos de chute para nos auxiliar com algumas estratégias/métodos, entre elas temos:

1. **Last**: Prevê o último valor 
2. **Mean**: Prevê a média do valor, dado um determinado período
3. **Drift**: Prevê os valores entre o primeiro e último valor, dado um determinado período

Nesse caso, iremos utilizar o método de prever o último valor (last).

### 4.1. Separando em treino e teste

Como mencionado acima, os bancos de dados serem provindos de uma competição do Kaggle, onde temos bancos de dados de treino e teste, os dados de teste são específicos para a submissão na competição e não possuem informação para comparar ambos os modelos, tanto o modelo de chute quanto o oficial. 

Devido a isso, iremos utilizar somente as informações do banco de dados de `sales_train`. Apesar daqueles pequenos bancos de dados fazerem parte, de algum jeito, do `sales_train`, eles não entrarão nessa primeira análise da criação do modelo inicial.

Séries temporais, devido a sua natureza sequencial de informações, não devem ser *splitadas* aleatoriamente, logo, devemos fazer essa separação adequada. Graças a biblioteca `sktime` temos 

In [28]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [29]:
sales_train[["shop_id", "item_id"]].value_counts()

shop_id  item_id
28       20949      867
53       20949      863
31       20949      863
22       20949      848
42       20949      834
                   ... 
56       3903         1
22       21608        1
         21605        1
         21597        1
46       18940        1
Length: 424124, dtype: int64

In [30]:
sales_train[(sales_train["shop_id"] == 28) & (sales_train["item_id"] == 20949)].date.min()

'01.01.2014'

# Criar a função que converte a data do dataframe